In [92]:
###apicilient

import requests
import time
import sys
import threading

def get_new_data(patient_id):
    res = requests.get(f"http://tesla.iem.pw.edu.pl:9080/v2/monitor/{patient_id}")
    js = res.json()
    
    return {
        "timestamp":js["trace"]["id"],
        "values": [x["value"] for x in js["trace"]["sensors"]],
        "anomalies":[x["anomaly"] for x in js["trace"]["sensors"]]
        }
 

In [93]:
###app

stop_collector = False
class DataCollectorThread(threading.Thread):
    def run(self):
        store = get_storage()
        for i in range(200):
            add_measurements("2", get_new_data("2"))
            expire_data(5) 
            time.sleep(1)
            
            if stop_collector:
                print("Stopping on request")
                break

if __name__ == "__main__":
   # print(get_new_data("2"))
   # sys.exit(0)


    init_storage()
    
    collector = DataCollectorThread()
    collector.start()
    #print("Waiting...")
    #time.sleep(7)
    
    stop_collector = True
    collector.join()
    print("Finished") 


Stopping on request
Finished


In [94]:
### storage

def init_storage():
    global _storage
    _storage = {}
    return _storage

def get_storage():
    global _storage
    return _storage
    

def add_measurements(patient_id, data):
    st = get_storage()
    if patient_id not in st:
        pd = {
            "patient_id": patient_id,
            "timestamps": [],
            "values": [],
            "anomalies": [],
            "_expires_ts":[]
        }
    else:
        pd = st[patient_id]
        
    pd["timestamps"].append(data["timestamp"])  
    pd["values"].append(data["values"])  
    pd["anomalies"].append(data["anomalies"])  
    pd["_expires_ts"].append(time.time())  
        

def expire_data(secs):
    st = get_storage()
    for pid, pd in st.items():
        ts = time.time()
        while len(pd["_expire_ts"]) > 0 and pd["_expire_ts"][0] < (ts-secs):
            pd["timestamps"].pop(0)
            pd["values"].pop(0)
            pd["anomalies"].pop(0)  
            pd["_expires_ts"].pop(0)

In [95]:
### dash.App

